In [61]:
#Gauss-Seidel method
#Works for SPD or strictly diagonally dominant matrices

import numpy as np

def GauSeiIT(A,u,f):
    N = np.size(u) 
    u[0] = (f[0] - np.dot(A[0,1:],u[1:]))/A[0,0] #Do first and last steps manually 
    for i in range(1,N-1): #range(a,b) = a,a+1,...,b-1
        # u[a:b] = [ u[a] , ... , u[b-1] ]
        u[i] = (f[i]-np.dot(A[i,:i],u[:i])-np.dot(A[i,i+1:],u[i+1:]))/A[i,i] #algorithm
    u[-1] = (f[-1]-np.dot(A[-1,:-1],u[:-1]))/A[-1,-1] #last step
    return u

def GauSei(A,u,f,n): #Do n steps of Gauss-Seidel
    u0 = u
    for i in range(n):
        #print(u0)
        u0 = GauSeiIT(A,u0,f)
    return u0


In [81]:
#SOR method

import numpy as np

def SORit(A,u,w,f):
    N = np.size(u)
    s0 = u[0]
    u[0] = (f[0] - np.dot(A[0,1:],u[1:]))/A[0,0]  #First step manually
    u[0] = (1-w)*s0+w*u[0]
    for i in range(1,N-1): #Do N-2 steps automatically following algorithm
        s = u[i]
        u[i] = (f[i]-np.dot(A[i,:i],u[:i])-np.dot(A[i,i+1:],u[i+1:]))/A[i,i]
        u[i] = (1-w)*s+w*u[i]
    sN = u[-1] #Do last step manually
    u[-1] = (f[-1]-np.dot(A[-1,:-1],u[:-1]))/A[-1,-1]
    u[-1] = (1-w)*sN+w*u[-1]
    return u

def SOR(A,u,w,f,N):
    u0 = u
    for i in range(N):
        u0 = SORit(A,u0,w,f)
    return u0

In [82]:
#SSOR
#Same method but with a forward and backwards SOR

def SORBWit(A,u,w,f): 
    N = np.size(u)
    sN = u[-1] #Do last step manually
    u[-1] = (f[-1]-np.dot(A[-1,:-1],u[:-1]))/A[-1,-1]
    u[-1] = (1-w)*sN+w*u[-1]
    for i in reversed(range(1,N-1)): #Do N-2 steps automatically following algorithm but reversed
        s = u[i]
        u[i] = (f[i]-np.dot(A[i,:i],u[:i])-np.dot(A[i,i+1:],u[i+1:]))/A[i,i]
        u[i] = (1-w)*s+w*u[i]
    s0 = u[0]
    u[0] = (f[0] - np.dot(A[0,1:],u[1:]))/A[0,0]  #First step manually
    u[0] = (1-w)*s0+w*u[0]
    return u

def SSOR(A,u,w,f,N):
    u0 = u
    for i in range(N):
        u0 = SORit(A,u0,w,f)
        u0 = SORBWit(A,u0,w,f)
    return u0

In [87]:
#For testing methods on SPD matrix
#Create random matrix and mupltiply with transpose to have SPD matrix:

msize = 5
M = np.random.rand(msize,msize)
A = np.dot(M,M.transpose()) + np.eye(msize)*msize #diagonally dominant to make it positive definite
#print(A)
usol = np.random.rand(msize)
print("solution: ",usol)
f = np.matmul(A,usol)

ugs = GauSei(A,np.zeros(msize),f,500)
usor = SOR(A,np.zeros(msize),0.5,f,30)
ussor = SSOR(A,np.zeros(msize),0.5,f,500)
solution = np.linalg.solve(A,f)

print("GS: ", ugs)
print("SOR: ", usor)
print("SSOR: ", ussor)
print("numpy: ", solution)


solution:  [0.69412415 0.07888342 0.72534008 0.48912666 0.1029857 ]
GS:  [0.69412415 0.07888342 0.72534008 0.48912666 0.1029857 ]
SOR:  [0.69412416 0.07888347 0.72534013 0.48912657 0.10298572]
SSOR:  [0.69412415 0.07888342 0.72534008 0.48912666 0.1029857 ]
numpy:  [0.69412415 0.07888342 0.72534008 0.48912666 0.1029857 ]


In [84]:
M = np.array([[16,4],[-1,10]])
b = np.array([3,10])
ugs = GauSei(M,[0,0],b,38)
usor = SOR(M,[0,0],0.5,b,38)
ussor = SSOR(M,[0,0],0.5,b,50)

print("GS: ", ugs)
print("SOR: ", usor)
print("SSOR: ", ussor)


GS:  [-0.06097560975609756, 0.9939024390243902]
SOR:  [-0.06097560976345765, 0.9939024390257305]
SSOR:  [-0.06097560975609756, 0.9939024390243902]


In [85]:
M = np.array([[4,-1,-6,0],[-5,-4,10,8],[0,9,4,-2],[1,0,-7,5]])
f = np.array([2,21,-12,-6])

ugs = GauSei(M,np.zeros(4),f,30)
usor = SOR(M,np.zeros(4),0.5,f,50)
ussor = SSOR(M,np.zeros(4),0.5,f,30)
solution = np.linalg.solve(M,f)

print("GS: ", ugs)
print("SOR: ", usor)
print("SSOR: ", ussor)
print("numpy: ", solution)

print("test f: ", np.dot(M,ussor))

GS:  [ 3.57995889e+25  9.48276313e+25 -1.94702176e+26 -2.79742964e+26]
SOR:  [ 3. -2.  2.  1.]
SSOR:  [-6.94490161e+13 -1.60754534e+14 -9.29427119e+13 -4.60699041e+13]
numpy:  [ 3. -2.  2.  1.]
test f:  [ 4.40614741e+14 -3.07723134e+14 -1.72642185e+15  3.50800447e+14]
